O problema da mochila
=====================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser consideradas na hora de resolver o problema.

O `problema da mochila` é um problema clássico de otimização onde devemos maximizar o valor dos itens colocados em uma mochila enquanto respeitamos a restrição do peso total dos itens (do contrário, a mochila rasga).

Uma forma de considerar restrições em algoritmos genéticos é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: o nosso objetivo é maximizar o valor dos itens na mochila, então temos um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse apenas isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i).
$$

No entanto, apenas essa função não resolve o problema! Precisamos considerar o limite de peso da mochila. Para isso, *penalizamos* a função objetivo considerando essa restrição:

$f=\begin{cases}
0 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}\mathrm{valor}(i) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$



## Objetivo



Utilizar um algoritmo genético para encontrar uma solução para o problema da mochila. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos. Considere que a mochila tem limite de peso de 5 quilogramas.



## Descrição do problema



No problema da mochila, você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila consegue carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido na mochila, porém que tenham um peso dentro da capacidade da mesma.



# Lista de itens disponíveis

In [1]:
ITENS = {
    "Capa da invisibilidade (não lavar com roupas coloridas)": {
        "valor": 299.99,
        "peso": 0.5,
    },
    "Chinelos mágicos feitos de nuvem (tamanho 42)": {
        "valor": 39.95,
        "peso": 1.2,
    },
    "Sacola de pipoca doce que nunca acaba (1 em cada 10 não tem sabor de nada)": {
        "valor": 10.99,
        "peso": 1.6,
    },
    "Controle universal (o botão de aumentar o ar condicionado está quebrado)": {
        "valor": 5.99,
        "peso": 0.5,
    },
    "Caixa com um milhão de cactos infláveis (disponível apenas na cor azul)": {
        "valor": 1.50,
        "peso": 3.6,
    },
    "Robô limpador de paredes (diagnosticado com depressão moderada)": {
        "valor": 42.50,
        "peso": 10.8,
    },
    "Réplica da espada do rei Artur (feita por monges míopes do Alasca)": {
        "valor": 12.50,
        "peso": 4.9,
    },
    "Saliva de um unicórnio de verdade (sabor levemente agridoce)": {
        "valor": 100.00,
        "peso": 0.02,
    },
    "Poção que faz todo alimento ter sabor de picles (dura 30 minutos)": {
        "valor": 2.99,
        "peso": 1.4,
    },
    "Conjunto de todos os conjuntos (será que ele contém ele mesmo?)": {
        "valor": 500.00,
        "peso": 15.0,
    },
}

## Resolução



In [2]:
import random
from pprint import pprint
from functools import partial
from itertools import product

from funcoes_6 import funcao_objetivo_pop_mochila
from funcoes_6 import populacao_cb as cria_populacao
from funcoes_6 import selecao_roleta_max as funcao_selecao
from funcoes_6 import cruzamento_uniforme as funcao_cruzamento
from funcoes_6 import mutacao_simples_cb as funcao_mutacao
from funcoes_6 import calcula_mochila
from funcoes_6 import funcao_objetivo_mochila

In [3]:
LIMITE = 5
NUM_ITENS = len(ITENS)
ORDEM_DOS_ITENS = list(sorted(ITENS.keys()))

TAMANHO_POPULACAO = 10
NUM_GERACOES = 50
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.05

In [4]:
funcao_objetivo = partial(funcao_objetivo_pop_mochila, 
                          itens=ITENS, 
                          ordem_dos_itens=ORDEM_DOS_ITENS, 
                          limite=LIMITE)

In [5]:
populacao = cria_populacao(TAMANHO_POPULACAO, NUM_ITENS)
pprint(populacao)

[[1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 1, 1, 0, 0, 0],
 [1, 0, 0, 0, 1, 1, 0, 0, 1, 1],
 [1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 0, 0],
 [0, 1, 1, 1, 1, 0, 1, 0, 1, 0],
 [0, 1, 0, 1, 1, 0, 1, 0, 1, 1],
 [0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 0, 1, 0, 1]]


In [6]:
hall_da_fama = []

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo(populacao)        
    selecionados = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao(proxima_geracao, CHANCE_DE_MUTACAO)
    
    # Atualização do hall da fama
    fitness = funcao_objetivo(proxima_geracao)
        
    maior_fitness = max(fitness)
    indice = fitness.index(maior_fitness)
    hall_da_fama.append(proxima_geracao[indice])    
    
    # Encerramento
    populacao = proxima_geracao

In [7]:
fitness = funcao_objetivo(hall_da_fama)
maior_fitness = max(fitness)
indice = fitness.index(maior_fitness)
melhor_individuo_observado = hall_da_fama[indice]

print()
print("Você deve pegar os seguintes itens:")

for i in range(NUM_ITENS):
    if melhor_individuo_observado[i] == 1:
        print("+", ORDEM_DOS_ITENS[i])

print()

peso_total, valor_total = calcula_mochila(melhor_individuo_observado, ITENS, ORDEM_DOS_ITENS)

print(
    f"Com isso, sua mochila terá o valor de {valor_total:.2f} reais "
    f"e peso de {peso_total:.2f} quilogramas."
)


Você deve pegar os seguintes itens:
+ Capa da invisibilidade (não lavar com roupas coloridas)
+ Chinelos mágicos feitos de nuvem (tamanho 42)
+ Controle universal (o botão de aumentar o ar condicionado está quebrado)
+ Poção que faz todo alimento ter sabor de picles (dura 30 minutos)
+ Saliva de um unicórnio de verdade (sabor levemente agridoce)

Com isso, sua mochila terá o valor de 448.92 reais e peso de 3.62 quilogramas.


In [8]:
if NUM_ITENS <= 10:
    melhor_fitness_ever = -float("inf")

    for candidato in product([0, 1], repeat=NUM_ITENS):
        fobj = funcao_objetivo_mochila(candidato, ITENS, ORDEM_DOS_ITENS, LIMITE)
        if fobj > melhor_fitness_ever:
            melhor_fitness_ever = fobj
            melhor_resposta_ever = candidato
            
    print()
    print("Você deve pegar os seguintes itens:")

    for i in range(NUM_ITENS):
        if melhor_resposta_ever[i] == 1:
            print("+", ORDEM_DOS_ITENS[i])

    print()

    peso_total, valor_total = calcula_mochila(melhor_resposta_ever, ITENS, ORDEM_DOS_ITENS)

    print(
        f"Com isso, sua mochila terá o valor de {valor_total:.2f} reais "
        f"e peso de {peso_total:.2f} quilogramas."
    )


Você deve pegar os seguintes itens:
+ Capa da invisibilidade (não lavar com roupas coloridas)
+ Chinelos mágicos feitos de nuvem (tamanho 42)
+ Controle universal (o botão de aumentar o ar condicionado está quebrado)
+ Sacola de pipoca doce que nunca acaba (1 em cada 10 não tem sabor de nada)
+ Saliva de um unicórnio de verdade (sabor levemente agridoce)

Com isso, sua mochila terá o valor de 456.92 reais e peso de 3.82 quilogramas.
